In [1]:
# import Pkg; Pkg.add(["SimpleGraphs","JSON","TikzGraphs","TikzPictures","Graphs","DataStructures"])
import JSON
import TikzGraphs, TikzPictures
using SimpleGraphs
using Graphs
using DataStructures

In [2]:
#(true,<reason>) if known true, (false,<reason>) if known false
#proof is a list of strings naming theorems, and then an integer giving the rough 'deepness' of this proof.
struct KnownType
    valid::Bool
    proof::Vector{String}
    priority::Int
end

struct LangPoset
    v::Set{String} #known classes in the poset
    e::Dict{Tuple{String,String}, KnownType} #set of facts (x,y) where x ⊆ y or not.
end
#create empty LangPoset
function LangPoset()
    return LangPoset(Set{String}(), Dict{Tuple{String,String}, KnownType}())
end
#create LangPoset on given classes, but no relations
function LangPoset(names::Vector{String})::LangPoset
    poset = LangPoset()
    for name = names
        addVertPoset!(poset, name)
    end
    return poset
end

import Base.in, Base.getindex, Base.setindex!
#for checking if a class / relation is in the Poset
in(x::String, poset::LangPoset) = in(x, poset.v)
in(xy::Tuple{String, String}, poset::LangPoset) = in(xy, keys(poset.e))
getindex(poset::LangPoset, x::String, y::String) = poset.e[x,y]
setindex!(poset::LangPoset, thm::KnownType, x::String, y::String) = (poset.e[x,y] = thm)

function addVertPoset!(P::LangPoset, name::String)::Nothing
    push!(P.v, name)
    
    #every item is a subset of itself, no theorems needed. Deepness -1.
    knowObj::KnownType = KnownType(true, [], -1)
    P[name, name] = knowObj
    return
end

#List of classes that contain v, x⊇v. 
function above(P::LangPoset, v::String)::Vector{String}
    if v ∉ P
        error("Class not in poset ", v)
    end
    els = filter(P.e) do pp
        lhs, rhs = pp.first
        known = pp.second
       (lhs == v) && (known.valid == true)
    end
    return map(x -> x[2], collect(keys(els)))
end

#List of classes that v contains, x⊆v. 
function below(P::LangPoset, v::String)::Vector{String}
    if v ∉ P
        error("Class not in poset ", v)
    end
    els =  filter(P.e) do pp
        lhs, rhs = pp.first
        known = pp.second
       (rhs == v) && (known.valid == true)
    end
    return map(x -> x[1], collect(keys(els)))
end

#List of classes that definitely do not contain v, x⊉v.
function notabove(P::LangPoset, v::String)::Vector{String}
    if v ∉ P
        error("Class not in poset ", v)
    end
    els =  filter(P.e) do pp
        lhs, rhs = pp.first
        known = pp.second
       (lhs == v) && (known.valid == false)
    end
    return map(x -> x[2], collect(keys(els)))
end

#List of classes that v definitely does not contain, x⊈v.
function notbelow(P::LangPoset, v::String)::Vector{String}
    if v ∉ P
        error("Class not in poset ", v)
    end
    els =  filter(P.e) do pp
        lhs, rhs = pp.first
        known = pp.second
       (rhs == v) && (known.valid == false)
    end
    return map(x -> x[1], collect(keys(els)))
end

#Are classes x and y equal in P?
function is_equal(P::LangPoset, x::String, y::String)::Bool
  return (x,y) ∈ P && P.e[x,y].valid &&
        (y,x) ∈ P && P.e[y,x].valid
end

#Are classes x and y definitely unequal in P?
function is_unequal(P::LangPoset, x::String, y::String)::Bool
  return ((x,y) ∈ keys(P.e) && !P.e[x,y].valid) ||
        ((y,x) ∈ keys(P.e) && !P.e[y,x].valid)
end

#List of classes that v is known equal to
function equal(P::LangPoset, v::String)::Set{String}
    if v ∉ P.v
        error("Class not in poset ", v)
    end
    return filter(P.v) do x
        is_equal(P, v, x)
    end
end

#List of classes that v is known unequal to
function unequal(P::LangPoset, v::String)::Set{String}
    if v ∉ P["v"]
        error("Class not in poset ", v)
    end
    return filter(P.v) do x
        is_unequal(P, v, x)
    end
end

function synth_thm(thm1::KnownType, thm2::KnownType)::KnownType
    @assert thm1.valid || thm2.valid
    valid = thm1.valid && thm2.valid
    proof = vcat(thm1.proof, thm2.proof)
    priority = max(thm1.priority, thm2.priority)
    return KnownType(valid, proof, priority)
end

synth_thm(thm1::KnownType, thm2::KnownType, thm3::KnownType)::KnownType = synth_thm(thm1, synth_thm(thm2, thm3))

#given a theorem that implies that x⊆y, try to add it in.
#if it's a higher deepness than existing knowledge, skip.
#If it's x !⊆ y, then set 'fact' to false.
#if it conflicts with given info, gives an error.
function add_thm!(P::LangPoset, x::String, y::String, thm::KnownType)::Nothing
    if (x,y) ∈ keys(P.e) && P.e[x,y].valid != thm.valid
        error("Conflicting info, that (x,y) has ",fact," but ",thm," says otherwise.")
    end
    
    if (x,y) ∈ keys(P.e)
        oldThm = P.e[x,y]
        if oldThm.priority < thm.priority
            return #already a lower priority one there, skip.
        end
        if (oldThm.priority == thm.priority) && (length(oldThm.proof) <= length(thm.proof))
            return #already have one of equal priority and no longer, skip.
        end
    end
    #otherwise add it in
    P.e[x,y] = thm
    return
end

function add_leq!(P::LangPoset, x::String, y::String, thm::String, priority::Int)
    known = KnownType(true, [thm], priority)
    
    #We learn that x⊆y. Then for and d⊆x and y⊆u, we know d⊆u.
    U = above(P,y)
    D = below(P,x)
  
    push!(U,y)
    push!(D,x)
    for u in U
        for d in D
            
            uThm = P.e[y,u]
            dThm = P.e[d,x]
            newThm = synth_thm(dThm, known, uThm)
            
            add_thm!(P, d, u, newThm)
        end
    end
end

function add_equals!(P::LangPoset, x::String, y::String, thm::String, priority::Int)
  add_leq!(P,x,y,thm,priority)
  add_leq!(P,y,x,thm,priority)
end

# ⊈
function add_notleq!(P::LangPoset, x::String, y::String, thm::String, priority::Int)
    known = KnownType(false, [thm], priority)
    
    #We learn that x⊈y, that is, there's an element of x not in y.
    #Then for and x⊆u and d⊆y, we know that element is in u and not d,
    #so u⊈d.
    U = above(P,x)
    D = below(P,y)
    
    push!(U,x)
    push!(D,y)
    for u in U
        for d in D
            
            uThm = P.e[x,u]
            dThm = P.e[d,y]
            newThm = synth_thm(dThm, known, uThm)
            
            add_thm!(P, u, d, newThm)
        end
    end
end

function interval(P::LangPoset, x::String, y::String)
  A = Set(above(P,x))
  B = Set(below(P,y))
  C = Set(below(P,x))
  D = Set(above(P,y))
  return collect(setdiff(intersect(A,B),union(C,D)))
end

interval (generic function with 1 method)

In [3]:
function prettyPrintClass(class)
  println(class["name"]," -- ",class["type"])
  println("  ",class["desc"])
  if "solver" in keys(class)
    println("  Def: ",class["solver"])
  end
  if "related" in keys(class)
    println("  See Also: ",join(class["related"],", "))
  end
end

function prettyPrint(db)
  for class = db["classes"]
    prettyPrintClass(class)
  end
end

prettyPrint (generic function with 1 method)

In [4]:
function make_language_poset(classlist=language_classes; verbose=false)
  #Assemble lattice of known inclusions and nonequalities
  #class_lattice = LanguagePoset(collect(keys(classlist)))
  class_lattice = LangPoset(collect(keys(classlist)))

  for (thmname, thm) = theorems
        
    #Simple 'priority' assignment based on the 'impliedby' and giving ObviousConstruction priority.
    rootThm = thm
    while "impliedby" ∈ keys(rootThm)
        rootThm = theorems[rootThm["impliedby"]]
    end
    if "priority" ∈ keys(rootThm)
        priority = rootThm["priority"]
    else
        priority = 1
    end
    
    content = thm["content"]
    if content == ""
      continue #no content implemented
    end
    if content[1] == '{'
      verbose && println("Skipping statement ",content)
      continue #depends on arguments, not implemented
    end
    #dead simple "parsing"
    for part = split(content,"&&")
      if part[1] == '{'
        verbose && println("Skipping statement ",part," from ",content)
        continue #depends on arguments, not implemented
      end
      rel = first.(String.(getfield.(collect(eachmatch(r"[⊂⊆⊃⊇⊈⊉⊊≠=]", part)), :match)))
      if rel === nothing
        println("Unknown statement ",part)
        continue
      end
      @assert length(rel) == 1 content*" has multiple operators "*string(rel)
      rel = rel[1]
      lhs, rhs = String.(split(part, rel))
      @assert lhs in keys(classes) lhs, "Language "*lhs*" not known, in theorem "*thmname
      @assert rhs in keys(classes) rhs, "Language "*rhs*" not known, in theorem "*thmname
      if lhs ∉ keys(classlist) && lhs ∈ keys(classes)
        if verbose
          println("Skipping ",part," for being wrong language type")
        end
        continue
      end
      if rel == '⊊' #normalize subsetnoteq
        rel = '⊂'
      end
      #normalize lhs->rhs ordering
      if rel == '⊃'
        rel = '⊂'
        lhs,rhs = rhs,lhs
      elseif rel == '⊇'
        rel = '⊆'
        lhs,rhs = rhs,lhs
      elseif rel == '⊉'
        rel = '⊈'
        lhs,rhs = rhs,lhs
      end
      #Use the operator to modify poset appropriately
        if rel == '⊆'
            verbose && println("Add ",lhs," ---> ",rhs)
            add_leq!(class_lattice, lhs, rhs, thmname, priority)
        elseif rel == '='
            verbose && println("Add ",lhs," <==> ",rhs)
            add_equals!(class_lattice, lhs, rhs, thmname, priority)
        elseif rel == '⊈'
            verbose && println("Add ",lhs," -/-> ",rhs)
            add_notleq!(class_lattice, lhs, rhs, thmname, priority)
        elseif rel == '⊂'
            verbose && println("Add ",lhs," -/-> ",rhs)
            add_leq!(class_lattice, lhs, rhs, thmname, priority)
            add_notleq!(class_lattice, rhs, lhs, thmname, priority)
        elseif rel == '≠'
            #do nothing, handled in is_neq
            println("Relation ≠ in ", thmname," should be replaced with ⊈ or ⊉.")
        else
            println("Strange relation ",rel)
        end
    end
  end
  #Now go through class equalities and normalize.
  return class_lattice
end

function LangPoset_to_SimpleDiGraph(poset; verbose=false)
  vertlist = collect(poset.v)
  v_to_i = v -> findfirst(x->x==v, vertlist)
  edge_list = Edge{Int}[]
  for v1 = vertlist
    i1 = v_to_i(v1)
    for v2 = above(poset, v1)
      if v1 == v2
        continue #skip self-loops
      end
      if v1 ∈ above(poset, v2)
        continue #skip equality
      end
      i2 = v_to_i(v2)
      if length(interval(poset, v1, v2)) > 0
        verbose && println("skip ",v1," to ",v2)
        continue
      end
      verbose && println("add ",v1," to ",v2)
      push!(edge_list, Graphs.SimpleEdge{Int}(i1,i2))
    end
    end
  SimpleDiGraph(edge_list), vertlist
end

function export_classes_for_web(class_lattice, langtype="Language")
    priority = 0 #TODO, for now all are equal priority
    
    #First filter out any classes that are equal to something else, except for the representatives of each
    equal_reps = Any[] #things to include
    equal_skips = Any[] #things to not
    for class_obj = db["classes"]
        if class_obj["type"] != langtype
            continue
        end
        class = class_obj["name"]
        
        #equal to something else?
        equals = collect(equal(class_lattice, class))
        if length(equals) > 1 #there's something else it's equal to
            canon_rep = intersect(equals, canonical_forms)
            if length(canon_rep) > 1
                error("Multiple representatives ",canon_rep," for equal classes ",equals)
            end
            if length(canon_rep) == 1
                canon_rep = canon_rep[1]
            else
                sort!(equals)
                canon_rep = equals[1]
                if canon_rep == class
                    println("Choosing ",class," as representative for ",equals)
                end
            end
            if canon_rep != class
                continue
            end
        end
        setdiff!(equals, [class]) #delete self from list of equals
        
        push!(equal_reps, class)
        append!(equal_skips, equals)
    end
    
    class_list = Any[]
    for class_obj = db["classes"]
        class = class_obj["name"]
        
        if class_obj["type"] != langtype #TODO: non-language classes
            continue
        end
        
        children = Any[]
        #only collect children if it's not skipped
        is_equal = class ∈ equal_skips
        if !is_equal
            
            for class2 = above(class_lattice, class) #loop over neighbors
                if class == class2 #skip over self
                    continue
                end
                if class2 ∈ equal_skips #skip over non-representative classes
                    continue
                end
                if length(interval(class_lattice, class, class2)) > 0 #skip over non-immediate children
                    continue
                end
                #TODO: add edge info on each child
                push!(children, class2)
            end
            
            equals = equal(class_lattice, class)
            setdiff!(equals, [class])
        
        else
            equals = false
            
        end

        desc = class_obj["desc"]
        related = class_obj["related"]
        properties = class_obj["properties"]
        notes = "notes" ∈ keys(class_obj) ? class_obj["notes"] : ""
        
        #TODO: "related", "notes", "properties"
        export_obj = Dict(
            "name"=>class,
            "desc"=>desc,
            "children"=>children,
            "equals"=>equals,
            "related"=>related,
            "notes"=>notes,
            "properties"=>properties
            )
        push!(class_list, export_obj)
    end
    
    class_list
end

export_classes_for_web (generic function with 2 methods)

In [5]:
#useful characters: ≤ ≥ ⊂ ⊆ ⊃ ⊇ ≠ = ∩ ⟹ Ω Σ Π Δ

s_raw = read("classes.json", String)
s = replace(s_raw, r"\n[ \t]*//.*" => "\n") #strip comments
db = JSON.parse(s);

#Verify the database makes reasonable sense
@assert all("name" in keys(t) for t in db["problem types"])
@assert all("desc" in keys(t) for t in db["problem types"])

@assert all("name" in keys(t) for t in db["properties"])
@assert all("desc" in keys(t) for t in db["properties"])

@assert all("name" in keys(t) for t in db["classes"])
@assert all("type" in keys(t) for t in db["classes"])
@assert all("desc" in keys(t) for t in db["classes"])

@assert all("name" in keys(t) for t in db["conjectures"])
@assert all("content" in keys(t) for t in db["conjectures"])

@assert all("name" in keys(t) for t in db["theorems"])
@assert all("content" in keys(t) for t in db["theorems"])

@assert all("name" in keys(t) for t in db["references"])
@assert all("desc" in keys(t) for t in db["references"])
@assert all("url" in keys(t) for t in db["references"])

problem_types = Dict((x->x["name"]=>x).(db["problem types"]))
classes = Dict((x->x["name"]=>x).(db["classes"]))
conjectures = Dict((x->x["name"]=>x).(db["conjectures"]))
theorems = Dict((x->x["name"]=>x).(db["theorems"]))
properties = Dict((x->x["name"]=>x).(db["properties"]))
references = Dict((x->x["name"]=>x).(db["references"]))

@assert all(t["type"] in keys(problem_types) for (n,t) in classes)

for (clsname, cls) = classes
    if "related" ∉ keys(cls)
        cls["related"] = Set{String}()
    end
    cls["related"] = Set{String}(cls["related"])
    for rlt = cls["related"]
        @assert (rlt ∈ keys(classes)) (clsname*" related to unknown class "*rlt)
    end
    
    if "properties" ∉ keys(cls)
        cls["properties"] = Set{String}()
    end
    cls["properties"] = Set{String}(cls["properties"])
    for prop = cls["properties"]
        @assert (prop ∈ keys(properties)) ("Unknown property "*prop)
    end
    
end

#Specifically classes that are languages (as opposed to e.g. promise problems)
language_classes = filter(x->x[2]["type"]=="Language", classes)
parametized_classes = filter(x->x[2]["type"]=="Parameterized Language", classes)

println("Initial checks passed")

Initial checks passed


In [6]:
#canonical names for classes that have equalities.
#if export_for_web talks about choosing canonical representatives, you can add them here to force one way.
canonical_forms = ["PSPACE","NC","L","NL","NLINSPACE","NC^0","RE","NEXP","SAC^1"];

class_lattice = make_language_poset(verbose=false);
param_lattice = make_language_poset(parametized_classes; verbose=false);

el,vl = LangPoset_to_SimpleDiGraph(class_lattice; verbose=false)
tikzplot = TikzGraphs.plot(el, vl, node_style="draw", graph_options="nodes={draw,circle}")
TikzPictures.save(TikzPictures.TEX("testgraph"), tikzplot)

In [7]:
println(class_lattice["ACC^0", "E"])
println(class_lattice["E","ACC^0"])
#prints: true => (["ACC^0⊆TC^0", "TC^0⊆NC^1", "NC^1⊆NC", "NC⊆P", "P⊆E"] => 1)
#        false => (["ACC^0⊆nuACC^0", "E⊈nuACC^0"] => 3)
#means: "true", ACC^0 is contained in E.
#       "false", E is not contained in ACC^0.
#and the reason: we know that ACC^0 is strictly contained in E, and E is strictly in EXP, which is enough. This uses the theorem
#named "E⊈nuACC^0", which has priority 3 (significantly nontrivial). But showing that ACC^0 ⊆ E only requires "simple" facts, so has
#priority 1.

println(class_lattice["PARITY","AC^0"])
#prints: false => (["PARITY⊈AC^0"] => 1)
#similar to the above, but we don't know that PARITY contains AC^0 (it doesn't); just that AC^0 doesn't contain PARITY.

println(class_lattice["NL","NP"])
#prints: true => (["NL⊆P", "P⊆NP"] => 1)
#means: "true", NL is contained in NP.
#and the reason: we know that NL is contained in P, and P is strictly in NP, which is enough.

println( ("BQP","NP") ∈ class_lattice)
println( ("NP","BQP") ∈ keys(class_lattice.e))
println( ("NP","P") ∈ keys(class_lattice.e))
println( ("NP","NL") ∈ keys(class_lattice.e))
#all print false: none of these are containments we know to be true or false.

KnownType(true, ["ACC^0⊆TC^0", "TC^0⊆NC^1", "NC^1⊆NC", "NC⊆P", "P⊆E"], 1)
KnownType(false, ["ACC^0⊆nuACC^0", "E⊈nuACC^0"], 3)
KnownType(false, ["PARITY⊈AC^0"], 3)
KnownType(true, ["NL⊆P", "P⊆NP"], 1)
false
false
false
false


In [8]:
open("langdata.json","w") do f
    json_export = export_classes_for_web(class_lattice)
    JSON.print(f, json_export)
end

open("paramlangdata.json","w") do f
    json_export = export_classes_for_web(param_lattice, "Parameterized Language")
    JSON.print(f, json_export)
end

open("references.json","w") do f
    json_export = references #no processing
    JSON.print(f, json_export)
end

In [59]:
prettyPrint(db)

A_0PP -- Language
  TODO. Defined in {ref:Vya03}.
  See Also: SBP, PP, AWPP
AC -- Language
  Unbounded Fanin Polylogarithmic-Depth Circuits. The class of decision problems solvable by a nonuniform family of Boolean circuits, with polynomial size, depth O(log^k(n)), and unbounded fanin, for some k. The gates allowed are AND, OR, and NOT. For a given k, we get AC^k, such as AC^1. The class AC^0 is when k=0 i.e. constant depth. Equal to NC and TC.
  See Also: NC, AC^0, AC^1, TC
AC^0 -- Language
  Unbounded Fanin Constant-Depth Circuits. An especially important subclass of AC, corresponding to constant-depth, unbounded-fanin, polynomial-size circuits with AND, OR, and NOT gates.
  See Also: NC, NC^0, AC^1, AC^0
AC^1 -- Language
  Unboudned Fanin Log-depth Circuits.
  See Also: AC, TC^1, NC^1, AC^0
ACC^0 -- Language
  AC0 With Arbitrary MOD Gates. Polynomial size, constant depth.
  See Also: AC^0, TC^0
ACKERMANN -- Language
  Ackermann function time. Problems solvable by a Turing machine in